In [ ]:
import os
from pathlib import Path
import json
import joblib
import numpy as np

In [5]:
import pandas as pd

# Cargar el dataset ya preparado
df = pd.read_csv("dataset_RN5.csv")

# Verifica columnas
print(df.columns)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_RN5.csv'

In [3]:
# Variables de entrada (todas menos la etiqueta)
X = df.drop(columns=['CLIMA_TIPO']).values

# Etiqueta de salida
y = df['CLIMA_TIPO'].values

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

# One-hot encoding de etiquetas
y_cat = to_categorical(y, num_classes=4)

# Dividir
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, shuffle=True)

# Escalar entradas
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model_clima = Sequential()
model_clima.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model_clima.add(Dropout(0.3))
model_clima.add(Dense(32, activation='relu'))
model_clima.add(Dropout(0.3))
model_clima.add(Dense(16, activation='relu'))
model_clima.add(Dense(4, activation='softmax'))  # 4 clases

model_clima.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_clima.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,124 (12.20 KB)

 Trainable params: 3,124 (12.20 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model_clima.fit(
    X_train_scaled, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test_scaled, y_test),
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - accuracy: 0.9811 - loss: 0.0486 - val_accuracy: 0.9870 - val_loss: 0.0292
Epoch 2/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.9814 - loss: 0.0477 - val_accuracy: 0.9885 - val_loss: 0.0281
Epoch 3/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9812 - loss: 0.0471 - val_accuracy: 0.9892 - val_loss: 0.0251
Epoch 4/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9820 - loss: 0.0456 - val_accuracy: 0.9862 - val_loss: 0.0276
Epoch 5/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9823 - loss: 0.0449 - val_accuracy: 0.9881 - val_loss: 0.0273


In [11]:
loss, acc = model_clima.evaluate(X_test_scaled, y_test)
print(f"📉 Loss: {loss:.4f}")
print(f"📈 Accuracy: {acc:.2%}")

2797/2797 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9892 - loss: 0.0251
📉 Loss: 0.0251
📈 Accuracy: 98.92%


In [2]:
def predecir_tipo_clima(variables, modelo, scaler):
    entrada = pd.DataFrame([variables])
    entrada_scaled = scaler.transform(entrada.values)
    pred = modelo.predict(entrada_scaled)[0]  # vector de probabilidades

    etiquetas = ['Soleado', 'Nublado', 'Ventoso', 'Lluvioso']
    clase = pred.argmax()

    print(f"🔍 Predicción: {etiquetas[clase]} ({clase})")
    print("\n📊 Probabilidades por clase:")
    for i, prob in enumerate(pred):
        print(f"  {etiquetas[i]} ({i}): {prob:.2%}")

    return clase, etiquetas[clase], pred

In [4]:
# Ejemplo con variables reales
variables = {
    'PRECTOTCORR': 4.04,
    'T2M_MAX': 30.69,
    'T2M_MIN': 23.48,
    'RH2M': 84.96,
    'WS10M': 1.10,
    'ALLSKY_SFC_SW_DWN': 19.36
}

predecir_tipo_clima(variables, model_clima, scaler)

NameError: name 'model_clima' is not defined

In [ ]:
# --- Guardar modelo RN5 automáticamente (ejecutar DESPUÉS de entrenar) ---
os.makedirs("models", exist_ok=True)

def _auto_save_rna5_artifacts(prefix="models"):
    g = globals()
    saved = {}
    posibles_modelos = ["model_clima", "model5", "model_rna5", "model_clasificador", "model"]
    posibles_scalers = ["scaler_clima", "scaler_rna5", "scaler_clas", "scaler"]
    for name in posibles_modelos:
        if name in g:
            joblib.dump(g[name], f"{prefix}/modelo_rna5.joblib")
            saved["model_clima"] = name
            break
    for name in posibles_scalers:
        if name in g:
            joblib.dump(g[name], f"{prefix}/scaler_rna5.joblib")
            saved["scaler_clima"] = name
            break
    return saved

# Ejecuta esto DESPUÉS de tu celda de entrenamiento en RN5
# saved = _auto_save_rna5_artifacts("models")
# print("Guardado automático RN5:", saved)

In [ ]:
# --- Funciones operativas para usar modelo RN5 (NO reentrena) ---

DEFAULT_MODEL_RNA5_PATH = "models/modelo_rna5.joblib"
DEFAULT_SCALER_RNA5_PATH = "models/scaler_rna5.joblib"
PREDICCIONES_RNA4_JSON = "data/predicciones_rna4.json"

def cargar_modelo_rna5(model_path=DEFAULT_MODEL_RNA5_PATH, scaler_path=DEFAULT_SCALER_RNA5_PATH):
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path) if Path(scaler_path).exists() else None
    return model, scaler

def predecir_tipo_clima_from_variables(variables, model_clima, scaler_clima=None, feature_order=None):
    if feature_order is None:
        feature_order = ['PRECTOTCORR','T2M_MAX','T2M_MIN','RH2M','WS10M','ALLSKY_SFC_SW_DWN']
    x = np.array([[float(variables.get(k, 0.0)) for k in feature_order]], dtype=float)
    if scaler_clima is not None:
        x = scaler_clima.transform(x)
    y = model_clima.predict(x)
    return y[0]

def clasificar_lugares(model_clima, scaler_clima=None, input_file=PREDICCIONES_RNA4_JSON):
    p = Path(input_file)
    if not p.exists():
        raise FileNotFoundError(f"Archivo de predicciones RN4 no encontrado en {input_file}")
    with p.open("r", encoding="utf-8") as f:
        predicciones = json.load(f)

    resultados = []
    for lugar in predicciones:
        variables = lugar.get("variables", {})
        try:
            etiqueta = predecir_tipo_clima_from_variables(variables, model_clima, scaler_clima)
        except Exception as e:
            etiqueta = "error"
        entry = dict(lugar)
        entry["clima"] = str(etiqueta)
        resultados.append(entry)
    return resultados

# %%
# EJEMPLO DE USO (tras haber guardado tus modelos):
# model5, sc5 = cargar_modelo_rna5()
# results = clasificar_lugares(model5, sc5)
# print(results[:2])